# Neural network for dose measurement

### Introduction and motivation
In modern dosimetry, the gold standard of radiation dose measurement is _effective dose_. This quantity is defined as a sum over dose to each major organ in the human body, weighted by a set of _tissue weighting factors_ that encode the average risk to health of a dose to that organ. This is a complex and difficult measurement to make, as many modern dosimeters are either optimised for field direction measurement or a fluence measurement. For example, for a Bonner sphere spectrometer, measuring effective dose is a time-consuming and computationally intensive process that requires careful measurement of the radiation field with varying size of polyethylene sphere and a complex unfolding algorithm to reconstruct the fluence, and this still lacks directional information about the field. 

In contrast, the segmentation of nFacet 3D encodes both the direction and the energy of an incident neutron field. This can be visualised through the direction and intensity of attenuation of neutron count in the detector cubes. There are therefore two components to measuring the effective dose: reconstructing the fluence of the incident neutron field, and the direction of incidence of the neutrons. Here I focus on the fluence reconstruction using an artificial neural network (ANN), as once trained this is a fast method for reconstructing the fluence without need for a complex unfolding algorithm. 

### Neural network architecture

The information for the network to learn is encoded in the distribution of neutron count across cubes in the detector. As a result, the first choice of input into the network consisted of 64 input neurons corresponding to the 64 cubes of the detector, whilst the output of the ANN was the binned fluence of the source. Additionally, summing the counts in planes of cubes provides additional information about the bulk response of the detector to the applied field and thus encodes more detailed information about the energy of the incident neutrons. This can be added to the inputs, adding 12 additional neurons corresponding to the four planes in the x, y and z directions. The model has subsequently been trained with and without these additional inputs to evaluate performance.

### Fluence binning scheme

TO BE ADDED

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
%matplotlib tk

import matplotlib.pyplot as plt
import NNPytorchLightning as NNPL

Here loading trained models to compare the training and validation losses per model, and look at performance on unseen data. These models are as follows:

model_cubes: trained just on cube counts, with a batch size of 200 and 200 samples per data set, learning rate of 1e-3, for 500 epochs (for time)

model_cubes_profiles: trained on cube counts and profiles, with a batch size of 200 and 200 samples per data set, learning rate of 1e-3, for 500 epochs (for time)

In [2]:
coeffs = '/home/nr1315/Documents/Project/effective_dose_coeffs.h5'
energy_bins = '/home/nr1315/Documents/Project/MachineLearning/energy_bins.npy'

model_cubes = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_new_data/version_1/',torch.rand((1,1,64)),coeffs,energy_bins)

model_cubes_profiles = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_profiles_new_data/version_5/',torch.rand((1,1,76)),coeffs,energy_bins)

Also need to load the loss curves separately, due to the way they are from the logs.

In [3]:
model_cubes_tloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_1-tag-train_loss.csv')
model_cubes_vloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_1-tag-val_loss.csv')
model_cubes_dose_err = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_1-tag-dose_err_AP.csv')
model_cubes_epoch = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_1-tag-epoch.csv')

model_cubes_profiles_tloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_5-tag-train_loss.csv')
model_cubes_profiles_vloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_5-tag-val_loss.csv')
model_cubes_profiles_dose_err = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_5-tag-dose_err_AP.csv')
model_cubes_profiles_epoch = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_5-tag-epoch.csv')

We also define the directories from which to load the testing data, for convenience:

In [4]:
testing_data_dir = '/home/nr1315/Documents/Project/MachineLearning/TestingData/'

AmBe_counts = 'SimCubeCounts_AmBe_5_0-0-0-0-1-0_1500.npy'
AmLi_counts = 'SimCubeCounts_AmLi_5_0-0-0-0-1-0_1500.npy'
Cf_counts = 'SimCubeCounts_Cf252_6_0-0-0-0-1-0_1500.npy'

AmBe_target = 'SimEnergyBins_AmBe.npy'
AmLi_target = 'SimEnergyBins_AmLi.npy'
Cf_target = 'SimEnergyBins_Cf252.npy'

For each model in turn, we will look at the loss curves and prediction on AmBe, AmLi, and Cf-252. 

### Cubes only model, 500 epochs, 200 samples per dataset



In [5]:
NNPL.PlotLosses([model_cubes_tloss,model_cubes_vloss],['Training loss','Validation loss'],'Cubes model, lr = 0.001, 200 samples per dataset',model_cubes_epoch)

/home/nr1315/miniconda3/envs/NewMLEnv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:623: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/nr1315/miniconda3/envs/NewMLEnv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:623: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


![Loss curves](Plots/model_cubes_loss.png)

Here the model seems to be training well, although it does not appear to finish training after 500 epochs. 

It is also informative to look at the quality of model prediction on unseen data, in this case AmLi, AmBe, and Cf252 sources. These can be seen below.

In [7]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

loss_fn = nn.MSELoss()

l1 = NNPL.compare_pred_true(model_cubes,testing_data_dir+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes only model',ax[0],24,np.load(energy_bins),1,loss_fn)
l2 = NNPL.compare_pred_true(model_cubes,testing_data_dir+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes only model',ax[1],24,np.load(energy_bins),1,loss_fn)
l3 = NNPL.compare_pred_true(model_cubes,testing_data_dir+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes only model',ax[2],24,np.load(energy_bins),1,loss_fn)

![Predictions of test data](Plots/model_cubes_prediction.png)

In [8]:
print("AmBe loss: {}".format(l1))
print("AmLi loss: {}".format(l2))
print("Cf-252 loss: {}".format(l3))

AmBe loss: 0.0878705158829689
AmLi loss: 0.04326992481946945
Cf-252 loss: 0.017681293189525604


Whilst the model loss continues to decrease, the model still has poor performance on the unseen sources,  predicting negative values in at least half the bins in all three cases. It generally seems to predict the greatest count in the region around the average energy of each source, i.e. above, below and at 1 MeV for AmBe, AmLi and Cf respectively, as previous iterations of the model did. One possible way to combat this may be to introduce more complex sources into the training set, such as linear combinations of existing monoenergetics, to help the network learn how to better reconstruct a more complicated fluence.

It is then informative to see the performance of the model on some of the validation data explicitly, as is shown below. The function used to plot this can be used to scroll through all of the training datasets.

In [9]:
from NNPytorchLightning import FluenceReconDataset, Resample

cubes_dataloader = torch.load('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_new_data/version_1/val_dloader.pt')
cubes_dataset,cubes_val_inds = cubes_dataloader.dataset.dataset,cubes_dataloader.dataset.indices

check = NNPL.CheckTrainData(model_cubes,cubes_dataset,np.load(energy_bins))

check.ViewTrainData()

![550keV model cubes pred](Plots/model_cubes_550keV_pred.png)

The model has generally performed well at predicting the bins for this validation data point with some count in an incorrect bin, but of note is that it is still predicting negative values in several bins. In order to avoid this, it may prove useful to add a term to the loss function that penalises any negative bins in the model prediction. 

A more thorough check of the validation data set for both this model and the following will be performed later.

### Cubes and profiles model, 200 samples per dataset, learning rate 0.001

In [10]:
NNPL.PlotLosses([model_cubes_profiles_tloss,model_cubes_profiles_vloss],['Training loss','Validation loss'],'Cubes and profiles model, lr = 0.001, 200 samples per dataset',model_cubes_profiles_epoch)

/home/nr1315/miniconda3/envs/NewMLEnv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:623: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/nr1315/miniconda3/envs/NewMLEnv/lib/python3.9/site-packages/scipy/interpolate/interpolate.py:623: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


![losses](Plots/model_cubes_profiles_200samples.png)

Whilst both the training loss and validation loss do both decrease, it is worth noting that the validation loss here is lower than the training loss, which in turn implies that the model needs to train for longer in this configuration. This is intuitive, as the increased number of bins due to the profile counts will mean there are a greater number of weights for the model to optimize. Other improvements may be found from increasing the learning rate or adding some learning rate scheduling, but otherwise training for longer is realistically required.

It is also worthy of note that the loss is higher than for the cubes-only model, but this may be a factor of the incomplete training. 

In [12]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

l4 = NNPL.compare_pred_true(model_cubes_profiles,testing_data_dir+'withProfiles/'+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes and \n profiles model',ax[0],24,np.load(energy_bins),1,loss_fn)
l5 = NNPL.compare_pred_true(model_cubes_profiles,testing_data_dir+'withProfiles/'+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes and \n profiles model',ax[1],24,np.load(energy_bins),1,loss_fn)
l6 = NNPL.compare_pred_true(model_cubes_profiles,testing_data_dir+'withProfiles/'+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes and \n profiles model',ax[2],24,np.load(energy_bins),1,loss_fn)

![cubes profiles 200 samples prediction](Plots/model_cubes_profiles_200samples_prediction.png)

In [13]:
print("AmBe loss: {}".format(l4))
print("AmLi loss: {}".format(l5))
print("Cf-252 loss: {}".format(l6))

AmBe loss: 0.09213663637638092
AmLi loss: 0.05291319638490677
Cf-252 loss: 0.04942954331636429


This model also performs poorly on the unseen data, although it has a lower loss than the cubes exclusive model. Most notably, this model predicts a large negative value in the first bin for all three sources, which further motivates introducing a penalty term to the loss to discourage any negative values in the model output. 


Validation data set checking needs some more work before conclusions can be drawn.

In [14]:
cubes_profiles_dataloader = torch.load('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_profiles_new_data/version_5/val_dloader.pt')
cubes_profiles_dataset,cubes_profiles_val_inds = cubes_profiles_dataloader.dataset.dataset,cubes_profiles_dataloader.dataset.indices

check = NNPL.CheckTrainData(model_cubes_profiles,cubes_profiles_dataset,np.load(energy_bins))

check.ViewTrainData()

After these results both models were trained again for 2000 epochs, as it appeared that neither model had fully trained in the 500 epochs here. These models and the monitoring quantities are loaded in here.

In [27]:
model_cubes_2000 = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_new_data/version_4/',torch.rand((1,1,64)),coeffs,energy_bins)

model_cubes_profiles_2000 = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_profiles_new_data/version_11/',torch.rand((1,1,76)),coeffs,energy_bins)

model_cubes_2000_tloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_4-tag-train_loss.csv')
model_cubes_2000_vloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_4-tag-val_loss.csv')
model_cubes_2000_dose_err = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_4-tag-dose_err_AP.csv')
model_cubes_2000_epoch = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_new_data_version_4-tag-epoch.csv')

model_cubes_profiles_2000_tloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_11-tag-train_loss.csv')
model_cubes_profiles_2000_vloss = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_11-tag-val_loss.csv')
model_cubes_profiles_2000_dose_err = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_11-tag-dose_err_AP.csv')

model_cubes_profiles_2000_epoch = pd.read_csv('/home/nr1315/Documents/Project/MachineLearning/LoggedParameters/run-model_cubes_profiles_new_data_version_11-tag-epoch.csv')

### Cubes model, learning rate 0.001, 200 samples per dataset, trained for 2000 epochs

In [29]:
NNPL.PlotLosses([model_cubes_2000_tloss,model_cubes_2000_vloss],['Training loss','Validation loss'],'Cubes model, lr = 0.001, 200 samples per dataset, 2000 epochs',model_cubes_2000_epoch,log=False)

![model cubes 2000epochs loss](Plots/model_cubes_2000_loss.png)

In [31]:
NNPL.PlotDoseError(model_cubes_2000_dose_err,'Cubes model, lr = 0.001, 200 samples per dataset, 2000 epochs, dose error',model_cubes_2000_epoch)

![model cubes 2000 dose error](Plots/model_cubes_2000_dose_err.png)

In [36]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

loss_fn = nn.MSELoss()

cubes_2000_AmBe_loss = NNPL.compare_pred_true(model_cubes_2000,testing_data_dir+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes only model,\n 2000 epochs',ax[0],24,np.load(energy_bins),1,loss_fn)
cubes_2000_AmLi_loss = NNPL.compare_pred_true(model_cubes_2000,testing_data_dir+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes only model,\n 2000 epochs',ax[1],24,np.load(energy_bins),1,loss_fn)
cubes_2000_Cf_loss = NNPL.compare_pred_true(model_cubes_2000,testing_data_dir+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes only model,\n 2000 epochs',ax[2],24,np.load(energy_bins),1,loss_fn)

![cubes model 2000 predictions](Plots/model_cubes_2000_prediction_1.png)

In [34]:
print("AmBe loss: {}".format(cubes_2000_AmBe_loss))
print("AmLi loss: {}".format(cubes_2000_AmLi_loss))
print("Cf-252 loss: {}".format(cubes_2000_Cf_loss))

AmBe loss: 0.0924101173877716
AmLi loss: 0.02471756935119629
Cf-252 loss: 0.034533705562353134


### Cubes and profiles model, learning rate 0.001, 200 samples per dataset, trained for 2000 epochs

In [19]:
NNPL.PlotLosses([model_cubes_profiles_2000_tloss,model_cubes_profiles_2000_vloss],['Training loss','Validation loss'],'Cubes and profiles model, lr = 0.001, 200 samples per dataset, 2000 epochs',model_cubes_profiles_2000_epoch,log=False)

![Cubes profiles 2000 epochs loss](Plots/model_cubes_profiles_2000epochs_loss.png)

In [23]:
NNPL.PlotDoseError(model_cubes_profiles_2000_dose_err,'Cubes and profiles model dose error',model_cubes_profiles_2000_epoch)

![cubes profiles 2000 epochs dose error](Plots/model_cubes_profiles_2000epochs_dose_err.png)

In [35]:
fig,ax=plt.subplots(1,3,figsize=(30,10))

cubes_profiles_2000_AmBe_loss = NNPL.compare_pred_true(model_cubes_profiles_2000,testing_data_dir+'withProfiles/'+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes and \n profiles model, 2000 epochs',ax[0],24,np.load(energy_bins),1,loss_fn)
cubes_profiles_2000_AmLi_loss = NNPL.compare_pred_true(model_cubes_profiles_2000,testing_data_dir+'withProfiles/'+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes and \n profiles model, 2000 epochs',ax[1],24,np.load(energy_bins),1,loss_fn)
cubes_profiles_2000_Cf_loss = NNPL.compare_pred_true(model_cubes_profiles_2000,testing_data_dir+'withProfiles/'+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes and \n profiles model, 2000 epochs',ax[2],24,np.load(energy_bins),1,loss_fn)

![cubes profiles 2000 predictions](Plots/model_cubes_profiles_2000_prediction.png)

In [37]:
print("AmBe loss: {}".format(cubes_profiles_2000_AmBe_loss))
print("AmLi loss: {}".format(cubes_profiles_2000_AmLi_loss))
print("Cf-252 loss: {}".format(cubes_profiles_2000_Cf_loss))

AmBe loss: 0.10277150571346283
AmLi loss: 0.05674799904227257
Cf-252 loss: 0.03377455845475197


### Current results & future steps

Prediction on the test data is approximately equivalently bad for all of these models. All models seem to have learned to predict significant values in at most 2 or 3 bins, likely as the vase majority of the sources used for training only have counts in at most 3 bins. A way of tackling this would be to augment the training dataset with sources that are continuous across more bins than the existing data points, which could be accomplished in several ways:

* Generating additional training data with a greater spread in energy (e.g. larger Gaussian spread), such that individual data points cover more energy bins
* Implementing a transform to produce new data points composed of a linear combination of two or more existing data points
* Training on 1 or 2 of the test sources, to see if that improves prediction on the third (and on more complex sources in general) 

Whilst the first approach is straightforward, it will add data points that peak around a particular bin and have a spread around that bin, rather than producing a more complex neutron spectrum that may have multiple peaks, although more complex distributions than a Gaussian could be used. It also would require generating a significant amount of additional training data, which is not storage efficient.

The second approach can in principle can teach the model more flexibility in how sources with a wide range of energies are encoded in the counts measured by the detector. This transformation would allow the model to learn an how an arbitrary fluence is encoded in the inputs, but in order to result in good prediction on real sources it may require some more careful logic on which sources are combined by the transformation, e.g. only taking those close in energy. However, following implementation of a rotation transform it may be informative to provide training data that is a composite of a high energy source on one side of the detector with a thermal source on the opposite side, to emulate a scenario with backscattered thermal neutrons. 

Regarding the final approach eventually the model will be trained on all of these sources, but in the mean time training on one of these may be useful to test the ability of the model to learn a more continuous source on top of the monoenergetic sources it is already learning.



The model also can regularly predicit negative values in energy bins, when obviously this is not a physical result. In order to prevent this, I propose adding a term to the loss that penalises any bins with a negative value in the output. This could for example be done with a simple boolean check on each value in the output tensor, adding a factor *k* to the loss for each bin that has a negative value. This factor *k* could be tuned at initialisation of the loss function in order to tune how harshly the model penalises negative bins, and the ideal value of this parameter can be investigated.

A final point of note is how many epochs the model takes to train - even after 2000 epochs the training and validation losses are still decreasing, along with the average absolute percentage error on dose. This motivates revisiting the optimisation of the network, particularly looking at the learning rate or a learning rate scheduler, as well as the choice of optimisation algorithm.